## Code objectives

- Get SWOT data in the region
- Create a grid and grid the data using spatial binning algorithm : First taking the mean at each grid cell, then grid including a time dimension
- Save the gridded data in an nc file

In [10]:
import numpy as np
import xarray as xr
import pandas as pd
import glob
import os
import functions
from scipy.interpolate import griddata

In [11]:
SwotData = "/Users/dihyachaal/Desktop/DataShelf/PassGroups/"
pass_dirs = sorted([d for d in glob.glob(os.path.join(SwotData, "*")) if os.path.isdir(d)])
bathy = xr.open_dataset('GEBCO_14_May_2025_b03991ce039e/gebco_2024_n75.0_s42.0_w-80.0_e-40.0.nc') 

## Creating a grid

In [20]:
res_km = 2.5 

latmin_study, latmax_study = 45, 54
lonmin_study, lonmax_study = -56, -50

mid_lat = (latmin_study + latmax_study) / 2 

deg_per_km_lat = 1 / 111.32 
deg_per_km_lon = 1 / (111.32 * np.cos(np.deg2rad(mid_lat)))

res_lat = res_km * deg_per_km_lat   
res_lon = res_km * deg_per_km_lon  

lon_bnd = np.arange(lonmin_study, lonmax_study+res_lon, res_lon)
lat_bnd = np.arange(latmin_study, latmax_study+res_lat, res_lat)

X, Y = np.meshgrid(lon_bnd, lat_bnd)

## Gridding the original data

- Spatial binning algorithm that aggregates
point measurements into a regular grid

In [27]:
# grid_sum = np.zeros((len(lat_bnd), len(lon_bnd)))
# ssh_sum = np.zeros((len(lat_bnd), len(lon_bnd)))
# ugos_sum = np.zeros((len(lat_bnd), len(lon_bnd)))
# vgos_sum = np.zeros((len(lat_bnd), len(lon_bnd)))
# grid_count = np.zeros((len(lat_bnd), len(lon_bnd)))
# ssh = []
# for p_dir in pass_dirs:
#     files = sorted(glob.glob(os.path.join(p_dir, "*.nc")))

#     for fp in files:
#         ds = xr.open_dataset(fp)
#         longitude = ((ds.longitude + 180) % 360) - 180
        
#         ds = ds.where((longitude >= lonmin_study) & 
#                               (longitude <= lonmax_study), drop = True)
        
#         ds = ds.where((ds.latitude >= latmin_study) & 
#                               (ds.latitude <= latmax_study), drop=True)
        
#         ds = drop_num_nadir(ds)
#         lon = ((ds.longitude + 180) % 360) - 180
#         lat = ds.latitude
#         ssh = ds['mdt'] + ds['ssha_unfiltered']
#         ugos = ds['ugos_filtered']
#         vgos = ds['vgos_filtered']
    
        
#        # 1D arrays
#         lon_flat = lon.values.flatten()
#         lat_flat = lat.values.flatten()
#         ssh_flat = ssh.values.flatten()
#         ugos_flat = ugos.values.flatten()
#         vgos_flat = vgos.values.flatten()

#         # Mask NaNs
#         mask = np.isfinite(lon_flat) & np.isfinite(lat_flat) & np.isfinite(ssh_flat) & np.isfinite(ugos_flat) & np.isfinite(vgos_flat)
#         lon_flat, lat_flat, ssh_flat, ugos_flat, vgos_flat = lon_flat[mask], lat_flat[mask], ssh_flat[mask], ugos_flat[mask], vgos_flat[mask]

#         # digitazing into bins
#         lon_idx = np.digitize(lon_flat, lon_bnd) - 1
#         lat_idx = np.digitize(lat_flat, lat_bnd) - 1

#         # keeping only valid bins
#         valid = (lon_idx >= 0) & (lon_idx < len(lon_bnd)) & \
#                 (lat_idx >= 0) & (lat_idx < len(lat_bnd))

#         lon_idx, lat_idx, ssh_flat, ugos_flat, vgos_flat = lon_idx[valid], lat_idx[valid], ssh_flat[valid], ugos_flat[valid], vgos_flat[valid]

        
#         np.add.at(ssh_sum, (lat_idx, lon_idx), ssh_flat)
#         np.add.at(ugos_sum, (lat_idx, lon_idx), ugos_flat)
#         np.add.at(vgos_sum, (lat_idx, lon_idx), vgos_flat)
        
#         np.add.at(grid_count, (lat_idx, lon_idx), 1)
        
#         ds.close()

# # Computing mean ssh at each grid cell
# ssh_mean = np.where(grid_count > 0, ssh_sum / grid_count, np.nan)
# ugos_mean = np.where(grid_count > 0, ugos_sum / grid_count, np.nan)
# vgos_mean = np.where(grid_count > 0, vgos_sum / grid_count, np.nan)

# # save to nc file
# out_ds = xr.Dataset(
#     {
#         "ssh": (("lat", "lon"), ssh_mean),
#         "count": (("lat", "lon"), grid_count),
#         "ugeos": (("lat", "lon"), ugos_mean),
#         "vgeos": (("lat", "lon"), vgos_mean)
        
#     },
#     coords={
#         "lat": lat_bnd,
#         "lon": lon_bnd
#     }
# )

# # out_ds.to_netcdf("gridded_swot_all_passes_2.5km.nc")

In [38]:
# times = []
# ugos_data = []
# vgos_data = []
# ssh_data = []
passes = []

gridded_products = []

for p_dir in pass_dirs:
    files = sorted(glob.glob(os.path.join(p_dir, "*.nc")))
    passes.append(files)
    for fp in files:
        ds = xr.open_dataset(fp)
        longitude = ((ds.longitude + 180) % 360) - 180
        
        ds = ds.where((longitude >= lonmin_study) & 
                              (longitude <= lonmax_study), drop = True)
        
        ds = ds.where((ds.latitude >= latmin_study) & 
                              (ds.latitude <= latmax_study), drop=True)
        
        ds = functions.drop_num_nadir(ds)
        lon = ((ds.longitude + 180) % 360) - 180
        lat = ds.latitude

    
        ssh = ds['mdt'] + ds['ssha_unfiltered']
        ugos = ds['ugos_filtered']
        vgos = ds['vgos_filtered']
        
        if 'time' in ds and ds['time'].size > 0:
            # time = ['time'].values.item() if ds['time'].size == 1 else ds['time'].values[0]
              time = ds['time'].values
              time = time[~np.isnat(time)]
        
       # 1D arrays
        lon_flat = lon.values.flatten()
        lat_flat = lat.values.flatten()
        ssh_flat = ssh.values.flatten()
        ugos_flat = ugos.values.flatten()
        vgos_flat = vgos.values.flatten()
        
        # Mask NaNs
        mask = np.isfinite(lon_flat) & np.isfinite(lat_flat) & np.isfinite(ssh_flat) & np.isfinite(ugos_flat) & np.isfinite(vgos_flat)
        lon_flat, lat_flat, ssh_flat, ugos_flat, vgos_flat = lon_flat[mask], lat_flat[mask], ssh_flat[mask], ugos_flat[mask], vgos_flat[mask]
        
        grid_sum = np.zeros((len(lat_bnd), len(lon_bnd)))
        ssh_sum = np.zeros((len(lat_bnd), len(lon_bnd)))
        ugos_sum = np.zeros((len(lat_bnd), len(lon_bnd)))
        vgos_sum = np.zeros((len(lat_bnd), len(lon_bnd)))
        grid_count = np.zeros((len(lat_bnd), len(lon_bnd)))

        # digitazing into bins
        lon_idx = np.digitize(lon_flat, lon_bnd) - 1
        lat_idx = np.digitize(lat_flat, lat_bnd) - 1
    
        valid = (lon_idx >= 0) & (lon_idx < len(lon_bnd)) & \
                (lat_idx >= 0) & (lat_idx < len(lat_bnd))
    
        lon_idx, lat_idx, ssh_flat, ugos_flat, vgos_flat = lon_idx[valid], lat_idx[valid], ssh_flat[valid], ugos_flat[valid], vgos_flat[valid]
        np.add.at(ssh_sum, (lat_idx, lon_idx), ssh_flat)
        np.add.at(ugos_sum, (lat_idx, lon_idx), ugos_flat)
        np.add.at(vgos_sum, (lat_idx, lon_idx), vgos_flat)
        np.add.at(grid_count, (lat_idx, lon_idx), 1)
        
        ssh_mean = np.where(grid_count > 0, ssh_sum / grid_count, np.nan)
        ugos_mean = np.where(grid_count > 0, ugos_sum / grid_count, np.nan)
        vgos_mean = np.where(grid_count > 0, vgos_sum / grid_count, np.nan)
    
        # ssh_data.append(ssh_mean)
        # ugos_data.append(ugos_mean)
        # vgos_data.append(vgos_mean)
        # times.append(time)

        gridded_product = xr.Dataset(
            {
                "ssh": (("lat", "lon"), np.array(ssh_mean)),
                "ugeos": (("lat", "lon"), np.array(ugos_mean)),
                "vgeos": (("lat", "lon"), np.array(vgos_mean))
                
            },
            coords={
                "time": time[0],
                "lat": lat_bnd,
                "lon": lon_bnd
            }
        )
        ds.close()    
        
        gridded_products.append(gridded_product)

/var/folders/7d/76c27lb97m18k6jd1mcn12yc0000gn/T/ipykernel_51562/97522983.py:66: RuntimeWarning: invalid value encountered in divide
  ssh_mean = np.where(grid_count > 0, ssh_sum / grid_count, np.nan)
/var/folders/7d/76c27lb97m18k6jd1mcn12yc0000gn/T/ipykernel_51562/97522983.py:67: RuntimeWarning: invalid value encountered in divide
  ugos_mean = np.where(grid_count > 0, ugos_sum / grid_count, np.nan)
/var/folders/7d/76c27lb97m18k6jd1mcn12yc0000gn/T/ipykernel_51562/97522983.py:68: RuntimeWarning: invalid value encountered in divide
  vgos_mean = np.where(grid_count > 0, vgos_sum / grid_count, np.nan)
/var/folders/7d/76c27lb97m18k6jd1mcn12yc0000gn/T/ipykernel_51562/97522983.py:66: RuntimeWarning: invalid value encountered in divide
  ssh_mean = np.where(grid_count > 0, ssh_sum / grid_count, np.nan)
/var/folders/7d/76c27lb97m18k6jd1mcn12yc0000gn/T/ipykernel_51562/97522983.py:67: RuntimeWarning: invalid value encountered in divide
  ugos_mean = np.where(grid_count > 0, ugos_sum / grid_coun

In [42]:
GriddedProd = xr.concat(gridded_products, dim='time')
GriddedProd.to_netcdf("gridded_swot_timeseries.nc")